This script is responsible for driving, works on Linux os.

In [ ]:
from Xlib import display, X
from PIL import Image
import cv2
import numpy as np
from joystick import LinuxVirtualJoystick
dsp = display.Display()
root = dsp.screen().root
roi = [2220,425,400,300]
gear_roi = [2220+539,428+90,8,8]

In [ ]:
def get_screen(roi):
    '''Get screen with Xlib from roi (startX, startY, width, heigh)'''
    raw = root.get_image(roi[0], roi[1], roi[2],roi[3], X.ZPixmap, 0xffffffff)
    image = Image.frombytes("RGB", (roi[2], roi[3]), raw.data, "raw", "BGRX")
    image = np.array(image.getdata(),dtype='uint8')\
    .reshape((image.size[1],image.size[0],3))
    return image

In [ ]:
def MeanShift(img):
    res = cv2.pyrMeanShiftFiltering(img, 1, 50)
    return res

import pickle

with open('km.pickle', 'rb') as f:
    """Loading pretrained k-means"""
    km = pickle.load(f)

def use_km(img, orig_img):
        #get indeces of non-ziro color
        indices = np.argwhere(img>1)
        """
        Cluster prediction
        Clusters 2 and 3 are marking lines, others are noise
        """
        try:
            """
            Gets the pixels belonging to the line clusters
            """
            lines = km.predict(indices)
            left_indx = np.argwhere(lines==1)
            right_indx = np.argwhere((lines==0) | (lines==2))
            """
            Get pixel coordinates of lines
            """
            left = np.take(indices, left_indx, axis=0).reshape(left_indx.shape[0], 2)
            right = np.take(indices, right_indx, axis=0).reshape(right_indx.shape[0], 2)
            """
            and fill them with color
            other pixels are set to zero
            """
            orig_img[:, :] = 0
            orig_img[right[:,0], right[:,1]] = 255
            orig_img[left[:,0], left[:,1]] = 255
            
        except:
            orig_img[:, :] =0
            
        return orig_img

def calc_dif(img, delta=1):
    gray = np.apply_along_axis(lambda x: np.average(x), 1, img)
    objects = []
    for idx, (x0, x1) in enumerate(zip(gray[:-1], gray[1:])):
        if (x1-x0)>delta:
            objects.append(idx)
    return objects

def img_proceed(roi):
    '''
    processing image: 
    resize->mean shift->gray->blur->trashhold->dilate->k-means
    The image is taken from Get_screen process
    '''
    orig_img = get_screen(roi)
        
    orig_img = cv2.resize(orig_img, (256,256))
    
    orig_img = MeanShift(orig_img)
        
    gray = cv2.cvtColor(orig_img, cv2.COLOR_RGB2GRAY)
        
    processed_img = cv2.GaussianBlur(gray, (5, 5), 0)
    processed_img = cv2.adaptiveThreshold(processed_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 25, -22)
    processed_img = cv2.dilate(processed_img, None, iterations = 1)
        
    #removing noise by using k-means
    processed_img = use_km(processed_img, processed_img)
        
    return processed_img


In [ ]:
joy = LinuxVirtualJoystick()
# multiply by constant to undo normalization
OUTPUT_NORMALIZATION = 32767

In [ ]:
def get_angle(predict, coef):
    """
    Set normalised wheel's angle,
    corrected by speed's coefficient
    """
    angle = predict
    angle *= OUTPUT_NORMALIZATION*(coef/100)
    try:
        return int(angle)
    except:
        return 0

In [ ]:
"""
x - gear number
y - wheel's coefficient 

"""
x = [7,8,9,10]
y = [200, 150, 145, 140]
z = np.polyfit(x, y, 3)
p = np.poly1d(z)

def set_coef(gear):
    return int(p(gear[0]))

In [ ]:
import pickle
with open('tree.pickle', 'rb') as f:
    """Loading pretrained pandom forest model for angle prediction"""
    tree = pickle.load(f)
    
with open('geartree.pickle', 'rb') as f:
    """Loading pretrained decision tree model for gear number recognition"""
    treec = pickle.load(f)

The main loop in which the image preprocessing function is called is recognized by the number of the gear engaged and the prediction of the model for the steering wheel angle is obtained.

In [ ]:
while 1:
    gear = get_screen(gear_roi)   
    gear = cv2.cvtColor(gear, cv2.COLOR_RGB2GRAY)
    coef = set_coef(treec.predict([gear.ravel()]))
    
    result_img = img_proceed(roi)
    res = cv2.resize(result_img, (128,128))
    res = res.reshape(1, 128*128)

    pred = tree.predict(res)

    angle = get_angle(pred[0], coef)

    joy.emit(angle, clutch=4000)
    cv2.imshow('frame', result_img)
    cv2.imshow('Gear', gear)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()